In [3]:
# load necessary libraries. install what you don't have
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
import time
#import io # new and important wow
#import keras ## new. install tensorflow?
#import argparse


In [ ]:
## Testing out the MET Museum API

In [ ]:
'''
EDA Questions to explore:

What is the frequency of European Artwork across the 1970-1990s period? 
If one decade dominates in terms of amount of pieces maybe we need to ensure
that we sample 1000 images from each decade for a more uniform sample

What is the frequency of drawings to photographs during the time period?
If there is an imbalance maybe we need to sample a set amount of each.

What is the average pixel dimensions of the images, based on this we have
justification for making each photo the same pixel size

How many NAs (year)


'''

In [15]:
def get_image_data(a, b): #input is in department number and department name
    url_test = "https://collectionapi.metmuseum.org/public/collection/v1"
    departments = ("/search?departmentId=" + str(a) +"&hasImages=true&dateBegin=1800&dateEnd=2000&q=" + b)
    response = requests.get(url_test+departments)
    if response.status_code == 200 :
        print("API request successful!")
        data = response.json()
    else:
        print("Failed to retrieve data: {response.status_code}")

    print("Total Items Retrieved From " + b + " department: " + str(data['total']))
    
    img_url = []
    date_list = []
    ID_list = data['objectIDs']
    
    index_val = 0 # if the distribution of years is wonky might want to consider random selection?
    counter_val = 0

    while counter_val <20: #set sample size
        object_id = ID_list[index_val]

        url_object = "https://collectionapi.metmuseum.org/public/collection/v1/objects/" 

        reponse_object = requests.get(url_object+str(object_id))

        if reponse_object.status_code == 200 :
            data_ob = reponse_object.json()

            image_url = data_ob['primaryImage']
            if image_url != "":
                img_url.append(image_url)
                date_val=data_ob['objectBeginDate']
                date_list.append(date_val)
            
                index_val+= 1
                counter_val+= 1
            else:
                index_val+=1
                counter_val=counter_val
            
            
            
            #conditional statement 

            #img_rep = requests.get(image_url).content
            #img = plt.imread(io.BytesIO(img_rep), format='JPG')
            #plt.imshow(img)

        # and this can be read directly as a np array i g? 
        else:
            print("Failed to retrieve url")
            index_val+= 1
            counter_val= counter_val 
            

        
        time.sleep(0.05)
    df = {'Year': date_list,
        'URL': img_url}
    df = pd.DataFrame(df)   
    df['Type'] = b
    return df
  

In [16]:
# code to show an image/process image characteristics.
'''
img_rep = requests.get(image_url).content
    img = plt.imread(io.BytesIO(img_rep), format='JPG')
    plt.imshow(img)
'''
# right now it is set to get 20 samples which is going to take 
# a few minutes in the first run. ideally we push to about 500? 
dr_url_list = get_image_data(9,"drawing")
pt_url_list = get_image_data(11,"painting")

print(len(dr_url_list), len(pt_url_list))


API request successful!
Total Items Retrieved From drawing department: 13423
Failed to retrieve url
Failed to retrieve url
Failed to retrieve url
Failed to retrieve url
Failed to retrieve url
API request successful!
Total Items Retrieved From painting department: 1014
20 20


In [17]:
pt_url_list.head()

,Year,URL,Type
0,1866,https://images.metmuseum.org/CRDImages/ep/orig...,painting
1,1804,https://images.metmuseum.org/CRDImages/ep/orig...,painting
2,1804,https://images.metmuseum.org/CRDImages/ep/orig...,painting
3,1846,https://images.metmuseum.org/CRDImages/ep/orig...,painting
4,1810,https://images.metmuseum.org/CRDImages/ep/orig...,painting
